***
##### This module defines the three Support Vector Regression (SVR) models. Each model is implemented with different kernel functions, which include linear kernel function, polynomial kernel function, and radial basis fucntion. After training the three SVR models, this module utilizes the models to do the stock price prediction.
 
##### Authored by Menghong Huang and Zhaosheng Li.
***

## Import Dependencies

In [4]:
%matplotlib inline
import numpy as np
from sklearn.svm import SVR     # Support Vector Regression
import matplotlib.pyplot as plt 
import pandas as pd             # For loading the data into dataframe

# For calculating RMSE, the usage refers to https://stackoverflow.com/a/18623635 
from sklearn.metrics import mean_squared_error
from math import sqrt

# For calculating the coefficient of determination (R^2), 
# the usage refers to https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
from sklearn.metrics import r2_score

## Load data into dataframe

In [38]:
df = pd.read_csv("data/GME_stock.csv")

In [39]:
df.head()

,date,open_price,high_price,low_price,close_price,volume,adjclose_price
0,2002-02-13,9.625,10.060,9.525,10.050,19054000,6.766666
1,2002-02-14,10.175,10.195,9.925,10.000,2755400,6.733003
2,2002-02-15,10.000,10.025,9.850,9.950,2097400,6.699336
3,2002-02-19,9.900,9.900,9.375,9.550,1852600,6.430017
4,2002-02-20,9.600,9.875,9.525,9.875,1723200,6.648838


In [40]:
df.tail()

,date,open_price,high_price,low_price,close_price,volume,adjclose_price
4768,2021-01-22,42.590000,76.760002,42.320000,65.010002,196784300,65.010002
4769,2021-01-25,96.730003,159.179993,61.130001,76.790001,177874000,76.790001
4770,2021-01-26,88.559998,150.000000,80.199997,147.979996,178588000,147.979996
4771,2021-01-27,354.829987,380.000000,249.000000,347.510010,93396700,347.510010
4772,2021-01-28,265.000000,483.000000,112.250000,193.600006,58815800,193.600006


## Split the dataframe into training set and test set

In [52]:
training_data = df[0:3773]  # from index 0 to 3772, NOT 3773
test_data = df[3773:4773]   # from index 3773 to 4773

## Build SVR models

## Train the SVR models